# Capstone Project - The Battle of Electronic Stores 

## Table of Contents 
* [Business Problem](#intro)
* [Data](#data)

## Business Problem <a name="intro"></a>

This project is geared toward stakeholders particularly interested in identifying an optimal location for a **computer electronics store** business in Los Angeles, California. The business is aimed at computer enthusiast and professional online gamers, thus providing a large variety of computers, computer parts, electronics, softwares, and gaming supplies. 

We will focus our attention on locations which **do not have more than two electronics stores in the area and are not primarly residential neighborhoods**. We would also prefer to consider locations which **contain malls, shopping centers, and a large population of retail stores**. 

Each location will then be recommended based on how close it satifies the above criteria along with a description of the advantages of choosing that location versus other candidates.  

## Data <a name="data"></a>

The data used consists of the following:

* Names of neighborhoods and regions within Los Angeles County were obtained through web scraping the Los Angeles Times' Mapping LA project. This data has been previously used by the City of Los Angeles Open Data portal to map out neighborhood boundaries.

* We make use of the neighborhood names data to obtain the longitude and latitude coordinates of each neighborhood using Geocoder ArcGIS API. 

* We use the Fouresquare API to obtain the most common venues in each nieghborhood.

* The number of electronic stores within each neighborhood will also be used to determine if a location will be recommended to our stakeholders or not. 